# <center>Introduction to Spark MLlib with Python</center>
## <center>Linear Regression and Classification</center>
### <center>July 20, 2016</center>

<img src = "https://ibm.box.com/shared/static/wfbduwkbx22nx3i2psbp9g27s2p9s86v.png", width="500" align = 'center'>

## <b>Welcome to the second lab in the course, Machine Learning with Spark MLlib.</b>
### <b>Spark has many libraries, namely under MLlib (Machine Learning Library)! Spark allows for quick and easy scalability of practical machine learning!</b>

In this lab exercise, you will learn about how to build a Linear Regression Model, a SVM model, and a Logistic Regression Model. 

### Some Notebook Commands
#### In case you haven't dealt with a Jupyter Notebook before, here are some quick, useful commands that may be handy to get started.
<ul>
    <li>Run a cell: CTRL + ENTER</li>
    <li>Create a cell above a cell: a</li>
    <li>Create a cell below a cell: b</li>
    <li>Change a cell to Markdown: m</li>
    
    <li>Change a cell to code: y</li>
</ul>

<b> If you are interested in more keyboard shortcuts, go to Help -> Keyboard Shortcuts </b>

## Linear Regression

<img src = "http://www.biostathandbook.com/pix/regressionlollipop.gif", style="height: 200pt; width: 200pt;", align = 'center'>
<div align="justify"><font size="3">Linear regression uses a "line of best fit", based on previous data in order to predict future values. There are plenty of model evaluation metrics that can be applied to linear regression. In this lab, we will look at <b>Mean Squared Error (MSE)</b></font></div>

Import the following libraries: <br>
<ul>
    <li>LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel from pyspark.mllib.regression</li>
</ul>

In [ ]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel

Now we need to create a <b>RDD of data</b> called <b>rdd_data</b>. That will be done by using the SparkContext (sc) to read in the <b>brain_body_data.csv</b> dataset. Take a look at the dataset so you have a feel for how it's structured.

In [ ]:
rdd_data = sc.textFile("/resources/data/brain_body_data.csv")

Now, run a <b>map function</b> on <b>rdd_data</b>, where the input is a <b>lambda function</b> that is as follows: <i>lambda line: line.split(",")</i>. This is so we can split the dataset by commas, since it's a comma-separated value file (CSV). Store this into a variable called <b>split_data</b>

In [ ]:
split_data = rdd_data.map(lambda line: line.split(","))

Next, run the following function that will convert each line in our RDD into a LabeledPoint.

In [ ]:
def labeledParse(line):
    return LabeledPoint(line[0], [line[1]])

Now, run a <b>map function</b> on <b>split_data</b>, passing in <b>labeledParse</b> as input. Store the output into a variable called <b>reg_data</b>.

In [ ]:
reg_data = split_data.map(labeledParse)

Now, we will create a variable called <b>linReg_model</b>, which will contain the linear regression model. The model will be made by calling the <b>LinearRegressionWithSGD</b> class and using the <b>.train</b> function with it. The .train function will take in 3 inputs:
<ul>
    <li>1st: The training data (reg_data in this case)</li>
    <li>2nd: The number of iterations, or how many times the regression will run (use iterations=150)</li>
    <li>3rd: step used in SGD (use step=0.00001 in this case) </li>
</ul>

In [ ]:
linReg_model = LinearRegressionWithSGD.train(reg_data, iterations=150, step=0.00001)

Next, we will create a variable called <b>actualAndPred</b>, which will contain the actual response, along with the predicted response from the model. This will be done by using the <b>map</b> function on <b>reg_data</b>, and passing in:<br> <b>lambda p: (p.label, linReg_model.predict(p.features))</b> as the input.

In [ ]:
actualAndPred = reg_data.map(lambda p: (p.label, linReg_model.predict(p.features)))

We will calculate the Mean Squared Error (MSE) value for the prediction. Run the following code to calculate the MSE. <br> <br> 

The map function takes the actual value and subtracts it by the predicted value, then 
squares the result. This is done for each value. <br> <br> 

Next, the reduce function sums all of the mapped values together. <br> <br>

Afterwards, the result is divided by the number of elements that are present in actualAndPreds.


In [ ]:
MSE = actualAndPred.map(lambda (v, p): (v - p)**2).reduce(lambda x, y: x + y) / actualAndPred.count()
print("Mean Squared Error = " + str(MSE))

---

## Support Vector Machine (SVM)

<img src = "http://blogs.quovantis.com/wp-content/uploads/2015/09/Svm_max_sep_hyperplane_with_margin.png", style="height: 200pt; width: 200pt;", align = 'center'>
<div align="justify"><font size="3">Support Vector Machines can be used for both <b>classification and regression</b> analysis. In our case, we will be using it for classification. Linear SVM in Spark only supports <b>binary classification</b>.</font></div>

Import the following libraries: <br>
<ul>
    <li>SVMWithSGD, SVMModel from pyspark.mllib.classification</li>
    <li>LabeledPoint from pyspark.mllib.regression</li>
</ul>

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import SVMWithSGD, SVMModel

Now we need to create a <b>RDD of data</b> called <b>svm_data</b>. That will be done by using the SparkContext (sc) to read in the <b>sample_svm_data.txt</b> dataset, which is a sample dataset that is built-in to Spark. It contains 322 rows of data. 

In [ ]:
svm_data = sc.textFile("/resources/sample_svm_data.txt")

For this dataset, it isn't in a format that we need, so we will need the following function to modify it. This function will also create LabeledPoints out of the data, which is necessary to train the SVM Model. Depending on your dataset, the parsing required will differ.

In [ ]:
def labeledParse(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

This will be applied to <b>svm_data</b> by using the <b>.map</b> function, and passing in the <b>labeledParse function</b>. This will apply the labeledParse function to the entire dataset. Call the output <b>svm_parsed</b>

In [ ]:
svm_parsed = svm_data.map(labeledParse)

Now create a SVM model using the <b>SVMWithSGD.train</b> function called <b>svm_model</b>, which requires two inputs:
<ul>
    <li>1st: The dataset containing the LabeledPoints (<b>svm_parsed</b> in this case)</li>
    <li>2nd: The number of iterations the model will run (<b>120</b> in this case)</li>
</ul>

In [ ]:
svm_model = SVMWithSGD.train(svm_parsed, iterations=120)

Next, we will create a variable called <b>svm_Labels_Predicts</b>, which will map a tuple containing the label and the prediction. <br>
This will be done by using the <b>.map</b> function once again, but on the parsed data, <b>svm_parsed</b>. <br>
The input into svm_parsed.map() will be a lambda function: <b>lambda x: (x.label, svm_model.predict(x.features))</b>

In [ ]:
svm_Labels_Predicts = svm_parsed.map(lambda p: (p.label, svm_model.predict(p.features)))

Now, we will take a look at the training error, called <b>trainingError</b>, which will tell us the accuracy of how well our model did. It will do this by counting the number of incorrect predictions it made, and divide it by the total number of predictions.<br>
We will run a <b>.filter</b> on the model we just created, <b>svm_Labels_Predicts</b>, <b>count</b> the output of that with <b>.count()</b>, then <b>divide</b> by the <b>number of elements in svm_parsed</b>. <br> <br>

This filter will take a lambda function as input: <b>lambda (v, p): v != p</b>, which just means that the function will look at the predicted value and the labeled value, then see if the prediction matched the label.<br><br>

Make sure to add a <b>.count()</b> to the <b>filter</b>, then <b>divide</b> the whole thing by <b>float(svm_parsed.count())</b>


In [ ]:
trainingError = svm_Labels_Predicts.filter(lambda (v, p): v != p).count() / float(svm_parsed.count()) 

Finally, print trainingError, to see the percentage that the model predicted incorrectly.

In [ ]:
print (trainingError)

---

## Logistic Regression

<img src = "http://cvxr.com/cvx/examples/cvxbook/Ch07_statistical_estim/html/logistics__01.png", style="height: 200pt; width: 200pt;", align = 'center'>
<div align="justify"><font size="3">Logistic Regression is a classifier, similar to SVM. Logistic Regression can be used for Binary Classification, which is pretty clear when looking at the diagram above. In the diagram, where are two distinct sections that data resides, which represents a binary classification. <br> <br> In this lab, we will use the same dataset as the one used for SVM, so we can compare the accuracy of both models.</font></div>

Import the following libraries: <br>
<ul>
    <li>LogisticRegressionWithLBFGS, LogisticRegressionModel from pyspark.mllib.classification</li>
    <li>LabeledPoint from pyspark.mllib.regression</li>
</ul>

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

Since we are still using the same dataset as in SVM, we will be using the same <b>svm_parsed</b> variable.

Create a variable called <b>logReg_model</b>, where we <b>train</b> a <b>LogisticRegressionWithLBFGS</b> model by passing in <b>svm_parsed</b>.

In [ ]:
logReg_model = LogisticRegressionWithLBFGS.train(svm_parsed)

Next, create a variable called <b>logReg_Labels_Predicts</b> by <b>mapping</b> the <b>svm_parsed</b> data and passing in the <b>label</b>, along with the <b>logReg_model prediction</b>. This is similar to what we did in the SVM section of the lab.

In [ ]:
logReg_Labels_Predicts = svm_parsed.map(lambda p: (p.value, logReg_model.predict(p.features)))

Finally, we will find the training error, or percentage that the model predicted incorrect. This will by done by applying the <b>filter</b> function on <b>logReg_Labels_Predicts</b>. We will pass in a lambda function that will filter for all values that do not equal <b>(lambda (v, p): v != p)</b>, then apply a <b>count()</b> on the filter. This will get the number of incorrect predictions. Now, we need to divide by the total number of predictions, or <b>float(svm_parsed.count())</b>. Store this as <b>trainingError2</b>. Refer to the SVM section if you need a hint.

In [ ]:
trainingError2 = logReg_Labels_Predicts.filter(lambda (v, p): v != p).count() / float(svm_parsed.count())

Now print trainingError2 and trianingError (from the SVM section)

In [ ]:
print trainingError2
print trainingError

It seems as though the training error for Logistic Regression is just slightly better than SVM for this case!